<a href="https://colab.research.google.com/github/bakhodirUsman/personalToDoApp/blob/main/nb/Sesame_CSM_(1B)-TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


[Vision RL](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) is now supported! Train Qwen2.5-VL, Gemma 3 etc. with GSPO or GRPO.

Introducing Unsloth [Standby for RL](https://docs.unsloth.ai/basics/memory-efficient-rl): GRPO is now faster, uses 30% less memory with 2x longer context.

Gpt-oss fine-tuning now supports 8× longer context with 0 accuracy loss. [Read more](https://docs.unsloth.ai/basics/long-context-gpt-oss-training)

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.52.3
!pip install --no-deps trl==0.22.2

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
from transformers import CsmForConditionalGeneration
import torch

model, processor = FastModel.from_pretrained(
    model_name = "unsloth/csm-1b",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Leave as None for auto-detection
    auto_model = CsmForConditionalGeneration,
    load_in_4bit = False, # Select True for 4bit - reduces memory usage
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Csm patching. Transformers: 4.52.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

unsloth/csm-1b does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.backbone_model` require gradients


In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)


Mounted at /content/drive


In [5]:
!unzip -q "/content/drive/MyDrive/tts_dataset.zip"

<a name="Data"></a>
### Data Prep  

We will use the `MrDragonFox/Elise`, which is designed for training TTS models. Ensure that your dataset follows the required format: **text, audio** for single-speaker models or **source, text, audio** for multi-speaker models. You can modify this section to accommodate your own dataset, but maintaining the correct structure is essential for optimal training.

In [48]:
# @title Dataset Prep for Google Drive (Unsloth / CSM-1B)
# Mount Drive and build a HF Dataset from your CSV + audio files.

import os, re, json, random
from pathlib import Path
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, Audio
from transformers import AutoProcessor


# 2) Point to your dataset folder on Drive
#    This folder should contain metadata files and the audio subfolders/files.
DATA_DIR = "/content/tts_dataset"   # <-- change to your folder
TARGET_SR = 24000

# 3) Find metadata files
data_dir = Path(DATA_DIR)
m_train = None
m_eval  = None
m_all   = None

if (data_dir / "metadata_train.csv").exists(): m_train = data_dir / "metadata_train.csv"
if (data_dir / "metadata_eval.csv").exists():  m_eval  = data_dir / "metadata_eval.csv"
if (data_dir / "metadata.csv").exists():       m_all   = data_dir / "metadata.csv"

if m_all and not (m_train and m_eval):
    # Try reading with different delimiters and error handling
    try:
        df = pd.read_csv(m_all, on_bad_lines='skip', quotechar='"')
    except Exception:
        try:
            df = pd.read_csv(m_all, sep=';', on_bad_lines='skip', quotechar='"')
        except Exception:
            try:
                df = pd.read_csv(m_all, sep='\t', on_bad_lines='skip', quotechar='"')
            except Exception as e:
                raise ValueError(f"Could not read metadata.csv with common delimiters. Error: {e}")

    if not {"path","text"}.issubset(df.columns):
        # Assuming the header is in the format 'audio_file|text|speaker_name'
        if len(df.columns) == 1 and "|" in df.columns[0]:
             df[['path', 'text', 'speaker_id']] = df.iloc[:, 0].str.split('|', expand=True)
        else:
            raise ValueError("metadata.csv must have 'path' and 'text' columns.")

    df = df.dropna(subset=["path","text"]).reset_index(drop=True)
    # Shuffle and split 90/10
    df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
    k = max(1, int(0.1 * len(df)))
    df_train, df_eval = df.iloc[:-k].copy(), df.iloc[-k:].copy()
else:
    if not (m_train and m_eval):
        raise FileNotFoundError("Provide either metadata.csv OR both metadata_train.csv and metadata_eval.csv")

    # Try reading train data with different delimiters and error handling
    try:
        df_train = pd.read_csv(m_train, on_bad_lines='skip', quotechar='"')
    except Exception:
        try:
            df_train = pd.read_csv(m_train, sep=';', on_bad_lines='skip', quotechar='"')
        except Exception:
            try:
                df_train = pd.read_csv(m_train, sep='\t', on_bad_lines='skip', quotechar='"')
            except Exception as e:
                raise ValueError(f"Could not read metadata_train.csv with common delimiters. Error: {e}")

    # Check and fix columns for df_train
    if not {"path","text"}.issubset(df_train.columns):
        if len(df_train.columns) == 1 and "|" in df_train.columns[0]:
             df_train[['path', 'text', 'speaker_id']] = df_train.iloc[:, 0].str.split('|', expand=True)
        else:
            raise ValueError(f"metadata_train.csv must have 'path' and 'text' columns.")

    # Try reading eval data with different delimiters and error handling
    try:
        df_eval  = pd.read_csv(m_eval, on_bad_lines='skip', quotechar='"')
    except Exception:
        try:
            df_eval  = pd.read_csv(m_eval, sep=';', on_bad_lines='skip', quotechar='"')
        except Exception:
            try:
                df_eval  = pd.read_csv(m_eval, sep='\t', on_bad_lines='skip', quotechar='"')
            except Exception as e:
                raise ValueError(f"Could not read metadata_eval.csv with common delimiters. Error: {e}")

    # Check and fix columns for df_eval
    if not {"path","text"}.issubset(df_eval.columns):
        if len(df_eval.columns) == 1 and "|" in df_eval.columns[0]:
             df_eval[['path', 'text', 'speaker_id']] = df_eval.iloc[:, 0].str.split('|', expand=True)
        else:
            raise ValueError(f"metadata_eval.csv must have 'path' and 'text' columns.")


    for name, df in [("train", df_train), ("eval", df_eval)]:
        if not {"path","text"}.issubset(df.columns):
            # This check should now be redundant due to the above checks, but kept for safety
            raise ValueError(f"metadata_{name}.csv must have 'path' and 'text' columns.")
        df.dropna(subset=["path","text"], inplace=True)


# 4) Make absolute paths
def to_abs(p: str) -> str:
    p = str(p).strip()
    # If already absolute, keep; else join with DATA_DIR
    return p if p.startswith("/") else str((data_dir / p).resolve())

df_train["audio_path"] = df_train["path"].apply(to_abs)
df_eval["audio_path"]  = df_eval["path"].apply(to_abs)

# 5) Add speaker/source column if missing (single-speaker default "0") and map string speaker IDs to integers
def ensure_speaker(df: pd.DataFrame) -> str:
    speaker_key = None
    if "source" in df.columns:
        speaker_key = "source"
    elif "speaker_id" in df.columns:
        speaker_key = "speaker_id"
    else:
        df["source"] = "0"
        speaker_key = "source"
        print('Unsloth: No speaker found, adding default "source" of "0".')

    # Map string speaker IDs to integers if they exist
    if speaker_key in df.columns and df[speaker_key].dtype == 'object':
        unique_speakers = df[speaker_key].unique()
        speaker_map = {speaker: str(i) for i, speaker in enumerate(unique_speakers)}
        df[speaker_key] = df[speaker_key].map(speaker_map)
        print(f"Unsloth: Mapped string speaker IDs to integers: {speaker_map}")

    return speaker_key

spk_key_train = ensure_speaker(df_train)
spk_key_eval  = ensure_speaker(df_eval)
assert spk_key_train == spk_key_eval, "Use the same speaker column name across splits."
speaker_key = spk_key_train

# 6) Build HF Datasets
#    We rename to match Unsloth expectations: columns -> audio (file path), text, source/speaker_id
def to_hfds(df: pd.DataFrame) -> Dataset:
    return Dataset.from_pandas(
        df[["audio_path", "text", speaker_key]].rename(columns={"audio_path": "audio"}),
        preserve_index=False
    )

raw_train = to_hfds(df_train)
raw_eval  = to_hfds(df_eval)

# 7) Cast the "audio" column to Audio() so 🤗 loads and resamples
raw_train = raw_train.cast_column("audio", Audio(sampling_rate=TARGET_SR))
raw_eval  = raw_eval.cast_column("audio",  Audio(sampling_rate=TARGET_SR))

raw_ds = DatasetDict(train=raw_train, eval=raw_eval)

# 8) Load Unsloth/CSM-1B processor
processor = AutoProcessor.from_pretrained("unsloth/csm-1b")

# 9) Preprocess function (mirrors your template, but reads from Drive)
#    IMPORTANT: processor.apply_chat_template expects the audio as a waveform array in the "audio" item.
def preprocess_example(example):
    # example["audio"] is a dict like {"path": "...", "array": np.ndarray, "sampling_rate": ...}
    # We pass the raw waveform array to the processor.
    conversation = [
        {
            "role": str(example[speaker_key]),
            "content": [
                {"type": "text",  "text": example["text"]},
                {"type": "audio", "path": example["audio"]["array"]},  # Unsloth’s processor reads the waveform here
            ],
        }
    ]
    try:
        model_inputs = processor.apply_chat_template(
            conversation,
            tokenize=True,
            return_dict=True,
            output_labels=True,
            text_kwargs = {
                "padding": "max_length",
                "max_length": 800,
                "pad_to_multiple_of": 8,
                "padding_side": "right",
            },
            audio_kwargs = {
                "sampling_rate": TARGET_SR,
                "max_length": 800000,   # Changed max_length to match the expected length from the error
                "padding": "max_length",
            },
            common_kwargs = {"return_tensors": "pt"},
        )
    except Exception as e:
        # If something breaks, print a short preview to help debug
        preview = example["text"][:80].replace("\n"," ")
        print(f"Error processing example: '{preview}...' -> {e}")
        return None

    required = ["input_ids", "attention_mask", "labels", "input_values", "input_values_cutoffs"]
    processed = {}
    for k in required:
        if k not in model_inputs:
            print(f"Warning: required key '{k}' missing from processor output.")
            return None
        processed[k] = model_inputs[k][0]  # remove batch dim

    # Final safety check
    if not all(isinstance(processed[k], torch.Tensor) for k in processed):
        print("Error: some outputs are not tensors:", {k: type(processed[k]) for k in processed})
        return None
    return processed

# 10) Map over both splits
processed_train = raw_ds["train"].map(
    preprocess_example,
    remove_columns=raw_ds["train"].column_names,
    desc="Preprocess (train)",
)
processed_eval = raw_ds["eval"].map(
    preprocess_example,
    remove_columns=raw_ds["eval"].column_names,
    desc="Preprocess (eval)",
)

processed_ds = DatasetDict(train=processed_train, eval=processed_eval)
print(processed_ds)

Unsloth: Mapped string speaker IDs to integers: {'yulduz': '0'}
Unsloth: Mapped string speaker IDs to integers: {'yulduz': '0'}


Preprocess (train):   0%|          | 0/3673 [00:00<?, ? examples/s]

Preprocess (eval):   0%|          | 0/919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'input_values', 'input_values_cutoffs'],
        num_rows: 3673
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'input_values', 'input_values_cutoffs'],
        num_rows: 919
    })
})


<a name="Train"></a>
### Train the model
Now let's use Huggingface  `Trainer`! More docs here: [Transformers docs](https://huggingface.co/docs/transformers/main_classes/trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [49]:
from transformers import TrainingArguments, Trainer
from unsloth import is_bfloat16_supported

trainer = Trainer(
    model = model,
    train_dataset = processed_ds["train"],
    eval_dataset = processed_ds["eval"],
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01, # Turn this on if overfitting
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-80GB. Max memory = 79.318 GB.
6.719 GB of memory reserved.


In [ ]:

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,673 | Num Epochs = 5 | Total steps = 2,300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,032,448 of 1,661,132,609 (1.75% trained)


Step,Training Loss
1,4.487100
2,4.194400
3,4.312700
4,4.284600
5,4.351200
6,4.327300
7,4.478600
8,4.382800
9,4.468500
10,4.414400


In [26]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

233.6955 seconds used for training.
3.89 minutes used for training.
Peak reserved memory = 6.719 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 8.471 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the prompts

In [1]:
from IPython.display import Audio, display
import soundfile as sf

text = "Ko‘chma savdo faoliyati bilan shug‘ullanuvchi tadbirkorlik sub’ektlari tomonidan elektron tarzda ko‘rib chiqish, vakolatli idora va tashkilotlar bilan kelishish hamda ro‘yxatdan o‘tkazish jarayonlarini yanada samarali tashkil etish maqsadida amaldagi 'K-Savdo‘ axborot tizimining yangi versiyasi ishlab chiqildi."
speaker_id = 0
inputs = processor(f"[{speaker_id}]{text}", add_special_tokens=True).to("cuda")
audio_values = model.generate(
    **inputs,
    max_new_tokens=140, # 125 tokens is 10 seconds of audio, for longer speech increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_without_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

NameError: name 'processor' is not defined

In [31]:
text = "Sesame judayam zo'r suniy idrok modeli."

speaker_id = 0
# Another equivalent way to prepare the inputs
conversation = [
    {"role": str(speaker_id), "content": [{"type": "text", "text": text}]},
]
audio_values = model.generate(
    **processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    ).to("cuda"),
    max_new_tokens=125, # 125 tokens is 10 seconds of audio, for longer speech increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_without_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

#### Voice and style consistency

Sesame CSM's power comes from providing audio context for each speaker. Let's pass a sample utterance from our dataset to ground speaker identity and style.

In [ ]:
speaker_id = 0

utterance = raw_ds[3]["audio"]["array"]
utterance_text = raw_ds[3]["text"]
text = "Sesame is a super cool TTS model which can be fine tuned with Unsloth."

# CSM will fill in the audio for the last text.
# You can even provide a conversation history back in as you generate new audio

conversation = [
    {"role": str(speaker_id), "content": [{"type": "text", "text": utterance_text},{"type": "audio", "path": utterance}]},
    {"role": str(speaker_id), "content": [{"type": "text", "text": text}]},
]

inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    )
audio_values = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=125, # 125 tokens is 10 seconds of audio, for longer text increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_with_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
processor.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# processor.push_to_hub("your_name/lora_model", token = "...") # Online saving

[]

### Saving to float16

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", processor, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", processor, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", processor, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", processor, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    processor.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    processor.push_to_hub("hf/model", token = "")


And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
